In [1]:
"""
Feedforward Circuit Simulation of the Pinksy-Rinzel Model and Wang and Buzsaki Model. Input current is given to both the cells. Synaptic conductance and delay is varied systematically.
Data file for every simulation is made and saved seperately for both somatic and dendritic inhibition as .mat files. 
"""

In [2]:
from brian2 import *
import scipy.io as mat
import numpy as np
import matplotlib.pyplot as plt

I_theory = mat.loadmat("D:\\Internship2\\feedforward_sims\\input_inhibitory_tau_50_sampling_20_kHz.mat")["input_theory_i"]
hidden_state = mat.loadmat("D:\\Internship2\\feedforward_sims\\input_inhibitory_tau_50_sampling_20_kHz.mat")["hidden_state_i"]
dt = 0.05*ms

In [3]:
# #For Somatic Inhibition run this cell
# def input_current():
#     """
#     Generates and returns the input current for the pyr_Iapp and int_Iapp variables.
#     """
#     pyr_scale = 20
#     pyr_baseline = -0.4
#     pyr_theory = I_theory.flatten()
#     pyr_val = pyr_scale*pyr_theory+pyr_baseline
#     pyr_Iapp = TimedArray(pyr_val*uA, dt=dt)

#     int_scale = 2
#     int_baseline = 0.2
#     int_theory = I_theory.flatten()
#     int_val = int_scale*int_theory+int_baseline
#     int_Iapp = TimedArray(int_val*uA, dt=dt)
#     return pyr_Iapp, int_Iapp

# def create_network(pyr_Iapp, int_Iapp, gsyn, delay, simtime):
#     """
#     Creates a network of PC and Int circuit and synapses for simulation.
#     Parameters:
#     - pyr_Iapp (function): Input current for PC
#     - int_Iapp (function): Input current for Int
#     - gsyn (siemens): Synaptic Conductance between interneuron and PC.
#     - delay (second): Synaptic Delay between interneuron and PC.
#     - simtime (second): Simulation time.
#     Returns:
#     - Pyr (StateMonitor): State monitor for PC
#     - Int (StateMonitor): State monitor for interneuron.
#     - Syn (StateMonitor): State monitor for synapse.
#     - S_int (SpikeMonitor): Spike monitor for interneuron.
#     - S_pyr (SpikeMonitor): Spike monitor for PC.
#     """

#     start_scope()
#     pyr_parameters = {
#     "gLs" : 0.1*mS,
#     "gLd" : 0.1*mS,
#     "gNa" : 30.*mS,
#     "gKdr" : 15.*mS,
#     "gCa" : 10.*mS,
#     "gKahp" : 0.8*mS,
#     "gKC" : 15.*mS,
#     "VNa" : 60.0*mV,
#     "VCa" : 80.0*mV,
#     "VK" : -75.0*mV,
#     "VL" : -60.0*mV,
#     "Cm" : 3.0*uF,
#     "gc" : 2.1*mS,
#     "pp" : 0.5,
#     "alphac": 2.0/ms,
#     "betac" : 0.1/ms,
#     "betaqd": 0.001/ms, 
#     }

#     pyr_initialvalues = {
#         'Vs' : -62.89223689*mV,
#         'Vd' : -62.98248752*mV,
#         'Cad' :0.21664282,
#         'hs' : 0.99806345,
#         'ns' :0.00068604,
#         'sd' : 0.01086703,
#         'cd' :0.00809387,
#         'qd' : 0.0811213,
#     }

#     int_parameters = {
#         "Cm" : 1*uF,
#         "iNa" : 35*msiemens,
#         "iK" : 9*msiemens,
#         "iL" : 0.1*msiemens,
#         "vNa" : 55*mV,
#         "vk" : -90*mV,
#         "vL" : -65*mV,
#     }

#     eqs_int = '''
#     dv_int/dt = (-Il - INa - Ik + I)/Cm : volt
#     Il = iL * (v_int - vL) : amp
#     INa = iNa * m**3 * h * (v_int - vNa) : amp
#     Ik = iK * n**4 * (v_int - vk) : amp
#     m = alpham / (alpham + betam) : 1
#     alpham = -0.1*(v_int/mV+35)/(exp(-0.1*(v_int/mV+35))-1)/ms : Hz
#     betam = 4*exp(-(v_int/mV+60)/18)/ms : Hz
#     dh/dt = 5 * (alphah * (1-h) - betah * h) : 1
#     alphah = 0.07*exp(-(v_int/mV+58)/20)/ms : Hz
#     betah = 1/(1+exp(-0.1*(v_int/mV+28)))/ms : Hz
#     dn/dt = 5 * (alphan * (1-n) - betan * n) : 1
#     alphan = -0.01*(v_int/mV+34)/(exp(-0.1*(v_int/mV+34))-1)/ms : Hz
#     betan = 0.125*exp(-(v_int/mV+44)/80)/ms : Hz

#     I = int_Iapp(t) : amp
#     '''

#     eqs_pyr = '''
#     #membrane potential equations
#     #for soma
#     dVs/dt=(-ILs-INa-IK_DR+(gc/pp)*(Vd-Vs)+(Is/pp)-Isyns)/Cm: volt

#     #for dendrite
#     dVd/dt=(-ILd-ICad-IK_ahp-IK_C+((gc/(1.0-pp))*(Vs-Vd))+Id_FN/(1.0-pp)-Isynd)/Cm: volt  #-Isynd/(1.0-pp)


#     ILs   = gLs*(Vs-VL) : amp
#     INa   = gNa*Minfs*Minfs*hs*(Vs-VNa) :amp
#     IK_DR = gKdr*ns*(Vs-VK) :amp


#     ILd    = gLd*(Vd-VL) :amp
#     ICad   = gCa*sd*sd*(Vd-VCa) :amp
#     IK_ahp = gKahp*qd*(Vd-VK) :amp
#     IK_C   = gKC*cd*chid*(Vd-VK) :amp

#     #calcium concentration
#     dCad/dt=  (-0.13*ICad/uA-0.075*Cad)/ms: 1

#     # soma dynamics 
#     dhs/dt= alphahs-(alphahs+betahs)*hs: 1 
#     dns/dt= alphans-(alphans+betans)*ns: 1

#     # dendrite dynamics
#     dcd/dt= alphacd-(alphacd+betacd)*cd: 1 
#     dqd/dt= alphaqd-(alphaqd+betaqd)*qd: 1

#     # calcium dynamics
#     dsd/dt= alphasd-(alphasd+betasd)*sd: 1

#     # soma state

#     # sodium current
#     alphams=  0.32*(-46.9-Vs/mV)/(exp((-46.9-Vs/mV)/4.0)-1.0)/ms: Hz
#     betams=  0.28*(Vs/mV+19.9)/(exp((Vs/mV+19.9)/5.0)-1.0)/ms: Hz #orignal one
#     # betams=  0.28*(Vs/mV+70)/(exp((Vs/mV+70)/5.0)-1.0)/ms: Hz

#     Minfs=    alphams/(alphams+betams): 1 


#     alphahs=  0.128*exp((-43.0-Vs/mV)/18.0)/ms: Hz
#     betahs=   4.0/(1.0+exp((-20.0-Vs/mV)/5.0))/ms: Hz


#     # Kdr potassium current
#     alphans=  0.016*(-24.9-Vs/mV)/(exp((-24.9-Vs/mV)/5.0)-1.0)/ms: Hz
#     betans=   0.25*exp(-1.0-0.025*Vs/mV)/ms: Hz

#     # dendrite state


#     # calcium induced potassium current
#     alphacd=((int(Vd/mV<=-10)*exp(((Vd/mV+50.0)/11) - (Vd/mV + 53.3) / 27))/18.975 + (int(Vd/mV>-10)*2.0*exp((-53.5-Vd/mV)/27.0)))/ms : Hz
#     betacd=(int(Vd/mV<=-10)*(2.0*exp((-53.5-Vd/mV)/27.0)-alphacd*ms))/ms : Hz

#     # AHP: calcium induced potassium current
#     alphaqd=  clip(0.00002*Cad,0.00002*Cad, 0.01)/ms: Hz
#     betaqd = 0.001/ms: Hz
#     chid= clip(Cad/250.,Cad/250. , 1. ) : 1 

#     # calcium current
#     alphasd=  (1.6/(1.0+exp(-0.072*(Vd/mV-5.0))))/ms: Hz
#     betasd=   (0.02*(Vd/mV+8.9)/(exp((Vd/mV+8.9)/5.0)-1.0))/ms: Hz
#     # betasd= (0.02*(Vd/mV+13)/(exp((Vd/mV+13)/5.0)-1.0))/ms: Hz

#     Is : amp
#     Id : amp
#     Id_FN = pyr_Iapp(t) : amp
#     Isyns : amp
#     Isynd : amp
#     '''

#     eqs_synapse = '''
#     Isyns_post = gsyn * ssyn * (Vs_post - vsyn) : amp  (summed)
#     dssyn/dt = (alpha_syn * (1 - ssyn)) * F - (beta_syn * ssyn) : 1 (clock-driven)
#     F = 1 / (1 + exp(-(v_int_pre - theta_syn) / sigma_syn)) : 1
#     alpha_syn = 1 / tau_rise : Hz 
#     beta_syn = 1 / tau_decay : Hz
#     gsyn : siemens
#     vsyn : volt
#     tau_rise : second
#     tau_decay : second
#     theta_syn : volt
#     sigma_syn : volt
#     '''

#     #neuron groups
#     pyr_model = NeuronGroup(1, model=eqs_pyr, threshold='Minfs > 0.5', refractory=2 * ms,
#                             dt=dt,method='euler',name='pyr',
#                             namespace=pyr_parameters) 
#     int_model = NeuronGroup(1, model=eqs_int, threshold='v_int > -20*mV', refractory=2 * ms,
#                             dt=dt, method='euler', name='int', 
#                             namespace=int_parameters)

#     #initial values
#     pyr_model.set_states(pyr_initialvalues)
#     pyr_model.Vs = pyr_initialvalues["Vs"]
#     pyr_model.Vd = pyr_initialvalues["Vd"]

#     int_model.v_int = -70*mV
#     int_model.h = 1
#     int_model.n = 0

#     #to make a conductance based synapse with interneron as presynaptic neuron and pyramidal neuron as post synaptic neuron
#     synapse_ff = Synapses(int_model, pyr_model, model=eqs_synapse, on_pre='ssyn += 1', method='euler', dt=dt, delay=delay)
#     synapse_ff.connect(i=0, j=0) 

#     synapse_ff.gsyn = gsyn
#     synapse_ff.vsyn = -62.0*mV
#     synapse_ff.tau_rise = 1.0*ms
#     synapse_ff.tau_decay = 9.0*ms 
#     synapse_ff.theta_syn = 0.0*mV
#     synapse_ff.sigma_syn = 1.0*mV

#     Pyr = StateMonitor(pyr_model, ['Vs','Vd','Cad','ILs',"INa",'ICad','IK_ahp','IK_C','ILd','Id_FN','Id'], record=True)
#     Int = StateMonitor(int_model, ['v_int','I'], record=True) # or record=0.1*ms
#     Syn = StateMonitor(synapse_ff,  ['ssyn', 'Isyns','gsyn'], record=True)

#     S_int = SpikeMonitor(int_model)
#     S_pyr = SpikeMonitor(pyr_model)

#     run(simtime)
#     return Pyr, Int, Syn, S_int, S_pyr

# def save_data(Pyr, Int, Syn, S_int, S_pyr, filename):
#     """
#     Save data from Pyr, Int, Syn, S_int, and S_pyr objects to .mat files.
#     """
#     data = {
#         'Pyr_Vs': Pyr.Vs[:],
#         'Pyr_Vd': Pyr.Vd[:],
#         'Pyr_Cad': Pyr.Cad[:],
#         'Pyr_ILs': Pyr.ILs[:],
#         'Pyr_INa': Pyr.INa[:],
#         'Pyr_ICad': Pyr.ICad[:],
#         'Pyr_IK_ahp': Pyr.IK_ahp[:],
#         'Pyr_IK_C': Pyr.IK_C[:],
#         'Pyr_ILd': Pyr.ILd[:],
#         'Pyr_Id_FN': Pyr.Id_FN[:],
#         'Pyr_Id': Pyr.Id[:],
#         'Int_v_int': Int.v_int[:],
#         'Int_I': Int.I[:],
#         'Syn_ssyn': Syn.ssyn[:],
#         'Syn_Isyns': Syn.Isyns[:],
#         'gsyn': Syn.gsyn[:],
#         'S_int_spikes': S_int.t[:],
#         'S_int_i': S_int.i[:],
#         'S_pyr_spikes': S_pyr.t[:],
#         'S_pyr_i': S_pyr.i[:],
#     }
#     mat.savemat(filename, data)

# def main():
#     """
#     Runs the simulation for different values of synaptic conductance and delay.
#     """
#     simtime = 180*second
#     pyr_Iapp, int_Iapp = input_current()
#     gsyn_list = [2,4,8]
#     delay_list = [0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5]
#     for gsyn in gsyn_list:
#         for delay in delay_list:
#             Pyr, Int, Syn, S_int, S_pyr = create_network(pyr_Iapp, int_Iapp, gsyn*mS, delay*ms, simtime)
#             save_name ="gsyn="+str(gsyn)+";delay="+str(delay)+".mat" 
#             print('saving data for gsyn=',gsyn,';delay=',delay)
#             save_data(Pyr, Int, Syn, S_int, S_pyr, save_name)

#     return Pyr, Int, Syn, S_int, S_pyr

# if __name__ == "__main__":
#     Pyr, Int, Syn, S_int, S_pyr = main()
#     print("Simulation done")

In [ ]:
# #For Dendritic Inhibition run this cell
# def input_current():
#     """
#     Generates and returns the input current for the pyr_Iapp and int_Iapp variables.
#     """
#     pyr_scale = 20
#     pyr_baseline = -0.4
#     pyr_theory = I_theory.flatten()
#     pyr_val = pyr_scale*pyr_theory+pyr_baseline
#     pyr_Iapp = TimedArray(pyr_val*uA, dt=dt)

#     int_scale = 2
#     int_baseline = 0.2
#     int_theory = I_theory.flatten()
#     int_val = int_scale*int_theory+int_baseline
#     int_Iapp = TimedArray(int_val*uA, dt=dt)
#     return pyr_Iapp, int_Iapp

# def create_network(pyr_Iapp, int_Iapp, gsyn, delay, simtime):
#     """
#     Creates a network of PC and Int circuit and synapses for simulation.
#     Parameters:
#     - pyr_Iapp (function): Input current for PC
#     - int_Iapp (function): Input current for Int
#     - gsyn (siemens): Synaptic Conductance between interneuron and PC.
#     - delay (second): Synaptic Delay between interneuron and PC.
#     - simtime (second): Simulation time.
#     Returns:
#     - Pyr (StateMonitor): State monitor for PC
#     - Int (StateMonitor): State monitor for interneuron.
#     - Syn (StateMonitor): State monitor for synapse.
#     - S_int (SpikeMonitor): Spike monitor for interneuron.
#     - S_pyr (SpikeMonitor): Spike monitor for PC.
#     """

#     start_scope()
#     pyr_parameters = {
#     "gLs" : 0.1*mS,
#     "gLd" : 0.1*mS,
#     "gNa" : 30.*mS,
#     "gKdr" : 15.*mS,
#     "gCa" : 10.*mS,
#     "gKahp" : 0.8*mS,
#     "gKC" : 15.*mS,
#     "VNa" : 60.0*mV,
#     "VCa" : 80.0*mV,
#     "VK" : -75.0*mV,
#     "VL" : -60.0*mV,
#     "Cm" : 3.0*uF,
#     "gc" : 2.1*mS,
#     # "gc" : 2.1mS, #for fig. 7: 3.0*mS, normal: 2.1*mS
#     "pp" : 0.5,
#     "alphac": 2.0/ms,
#     "betac" : 0.1/ms,
#     "betaqd": 0.001/ms, 
#     }

#     pyr_initialvalues = {
#         'Vs' : -62.89223689*mV,
#         'Vd' : -62.98248752*mV,
#         'Cad' :0.21664282,
#         'hs' : 0.99806345,
#         'ns' :0.00068604,
#         'sd' : 0.01086703,
#         'cd' :0.00809387,
#         'qd' : 0.0811213,
#     }

#     int_parameters = {
#         "Cm" : 1*uF,
#         "iNa" : 35*msiemens,
#         "iK" : 9*msiemens,
#         "iL" : 0.1*msiemens,
#         "vNa" : 55*mV,
#         "vk" : -90*mV,
#         "vL" : -65*mV,
#     }

#     eqs_int = '''
#     dv_int/dt = (-Il - INa - Ik + I)/Cm : volt
#     Il = iL * (v_int - vL) : amp
#     INa = iNa * m**3 * h * (v_int - vNa) : amp
#     Ik = iK * n**4 * (v_int - vk) : amp
#     m = alpham / (alpham + betam) : 1
#     alpham = -0.1*(v_int/mV+35)/(exp(-0.1*(v_int/mV+35))-1)/ms : Hz
#     betam = 4*exp(-(v_int/mV+60)/18)/ms : Hz
#     dh/dt = 5 * (alphah * (1-h) - betah * h) : 1
#     alphah = 0.07*exp(-(v_int/mV+58)/20)/ms : Hz
#     betah = 1/(1+exp(-0.1*(v_int/mV+28)))/ms : Hz
#     dn/dt = 5 * (alphan * (1-n) - betan * n) : 1
#     alphan = -0.01*(v_int/mV+34)/(exp(-0.1*(v_int/mV+34))-1)/ms : Hz
#     betan = 0.125*exp(-(v_int/mV+44)/80)/ms : Hz

#     I = int_Iapp(t) : amp
#     '''

#     eqs_pyr = '''
#     #membrane potential equations
#     #for soma
#     dVs/dt=(-ILs-INa-IK_DR+(gc/pp)*(Vd-Vs)+(Is_FN/pp)-Isyns)/Cm: volt

#     #for dendrite
#     dVd/dt=(-ILd-ICad-IK_ahp-IK_C+((gc/(1.0-pp))*(Vs-Vd))+Id/(1.0-pp)-Isynd)/Cm: volt  #-Isynd/(1.0-pp)


#     ILs   = gLs*(Vs-VL) : amp
#     INa   = gNa*Minfs*Minfs*hs*(Vs-VNa) :amp
#     IK_DR = gKdr*ns*(Vs-VK) :amp


#     ILd    = gLd*(Vd-VL) :amp
#     ICad   = gCa*sd*sd*(Vd-VCa) :amp
#     IK_ahp = gKahp*qd*(Vd-VK) :amp
#     IK_C   = gKC*cd*chid*(Vd-VK) :amp

#     #calcium concentration
#     dCad/dt=  (-0.13*ICad/uA-0.075*Cad)/ms: 1

#     # soma dynamics 
#     dhs/dt= alphahs-(alphahs+betahs)*hs: 1 
#     dns/dt= alphans-(alphans+betans)*ns: 1

#     # dendrite dynamics
#     dcd/dt= alphacd-(alphacd+betacd)*cd: 1 
#     dqd/dt= alphaqd-(alphaqd+betaqd)*qd: 1

#     # calcium dynamics
#     dsd/dt= alphasd-(alphasd+betasd)*sd: 1

#     # soma state

#     # sodium current
#     alphams=  0.32*(-46.9-Vs/mV)/(exp((-46.9-Vs/mV)/4.0)-1.0)/ms: Hz
#     betams=  0.28*(Vs/mV+19.9)/(exp((Vs/mV+19.9)/5.0)-1.0)/ms: Hz #orignal one
#     # betams=  0.28*(Vs/mV+70)/(exp((Vs/mV+70)/5.0)-1.0)/ms: Hz

#     Minfs=    alphams/(alphams+betams): 1 


#     alphahs=  0.128*exp((-43.0-Vs/mV)/18.0)/ms: Hz
#     betahs=   4.0/(1.0+exp((-20.0-Vs/mV)/5.0))/ms: Hz


#     # Kdr potassium current
#     alphans=  0.016*(-24.9-Vs/mV)/(exp((-24.9-Vs/mV)/5.0)-1.0)/ms: Hz
#     betans=   0.25*exp(-1.0-0.025*Vs/mV)/ms: Hz

#     # dendrite state


#     # calcium induced potassium current
#     alphacd=((int(Vd/mV<=-10)*exp(((Vd/mV+50.0)/11) - (Vd/mV + 53.3) / 27))/18.975 + (int(Vd/mV>-10)*2.0*exp((-53.5-Vd/mV)/27.0)))/ms : Hz
#     betacd=(int(Vd/mV<=-10)*(2.0*exp((-53.5-Vd/mV)/27.0)-alphacd*ms))/ms : Hz

#     # AHP: calcium induced potassium current
#     alphaqd=  clip(0.00002*Cad,0.00002*Cad, 0.01)/ms: Hz
#     betaqd = 0.001/ms: Hz
#     chid= clip(Cad/250.,Cad/250. , 1. ) : 1 

#     # calcium current
#     alphasd=  (1.6/(1.0+exp(-0.072*(Vd/mV-5.0))))/ms: Hz
#     betasd=   (0.02*(Vd/mV+8.9)/(exp((Vd/mV+8.9)/5.0)-1.0))/ms: Hz
#     # betasd= (0.02*(Vd/mV+13)/(exp((Vd/mV+13)/5.0)-1.0))/ms: Hz

#     Is_FN = pyr_Iapp(t) :amp
#     Is : amp
#     Id : amp
#     Isyns : amp
#     Isynd : amp
#     '''

#     eqs_synapse = '''
#     Isynd_post = gsyn * ssyn * (Vd_post - vsyn) : amp  (summed)
#     dssyn/dt = (alpha_syn * (1 - ssyn)) * F - (beta_syn * ssyn) : 1 (clock-driven)
#     F = 1 / (1 + exp(-(v_int_pre - theta_syn) / sigma_syn)) : 1
#     alpha_syn = 1 / tau_rise : Hz 
#     beta_syn = 1 / tau_decay : Hz
#     gsyn : siemens
#     vsyn : volt
#     tau_rise : second
#     tau_decay : second
#     theta_syn : volt
#     sigma_syn : volt
#     '''

#     #neuron groups
#     pyr_model = NeuronGroup(1, model=eqs_pyr, threshold='Minfs > 0.5', refractory=2 * ms,
#                             dt=dt,method='euler',name='pyr',
#                             namespace=pyr_parameters) 
#     int_model = NeuronGroup(1, model=eqs_int, threshold='v_int > -20*mV', refractory=2 * ms,
#                             dt=dt, method='euler', name='int', 
#                             namespace=int_parameters)

#     #initial values
#     pyr_model.set_states(pyr_initialvalues)
#     pyr_model.Vs = pyr_initialvalues["Vs"]
#     pyr_model.Vd = pyr_initialvalues["Vd"]

#     int_model.v_int = -70*mV
#     int_model.h = 1
#     int_model.n = 0

#     #to make a conductance based synapse with interneron as presynaptic neuron and pyramidal neuron as post synaptic neuron
#     synapse_ff = Synapses(int_model, pyr_model, model=eqs_synapse, on_pre='ssyn += 1', method='euler', dt=dt, delay=delay)
#     synapse_ff.connect(i=0, j=0) 

#     synapse_ff.gsyn = gsyn
#     synapse_ff.vsyn = -62.0*mV
#     synapse_ff.tau_rise = 5.0*ms
#     synapse_ff.tau_decay = 20.0*ms 
#     synapse_ff.theta_syn = 0.0*mV
#     synapse_ff.sigma_syn = 1.0*mV

#     Pyr = StateMonitor(pyr_model, ['Vs','Vd','Cad','ILs',"INa",'ICad','IK_ahp','IK_C','ILd','Is_FN','Is'], record=True)
#     Int = StateMonitor(int_model, ['v_int','I'], record=True) # or record=0.1*ms
#     Syn = StateMonitor(synapse_ff,  ['ssyn', 'Isyns','gsyn'], record=True)
#     # I_int = StateMonitor(int_model, 'I', record=True)

#     S_int = SpikeMonitor(int_model)
#     S_pyr = SpikeMonitor(pyr_model)

#     run(simtime)
#     return Pyr, Int, Syn, S_int, S_pyr

# def save_data(Pyr, Int, Syn, S_int, S_pyr, filename):
#     """
#     Save data from Pyr, Int, Syn, S_int, and S_pyr objects to .mat files.
#     """

#     data = {
#         'Pyr_Vs': Pyr.Vs[:],
#         'Pyr_Vd': Pyr.Vd[:],
#         'Pyr_Cad': Pyr.Cad[:],
#         'Pyr_ILs': Pyr.ILs[:],
#         'Pyr_INa': Pyr.INa[:],
#         'Pyr_ICad': Pyr.ICad[:],
#         'Pyr_IK_ahp': Pyr.IK_ahp[:],
#         'Pyr_IK_C': Pyr.IK_C[:],
#         'Pyr_ILd': Pyr.ILd[:],
#         'Pyr_Is_FN': Pyr.Is_FN[:],
#         'Pyr_Is': Pyr.Is[:],
#         'Int_v_int': Int.v_int[:],
#         'Int_I': Int.I[:],
#         'Syn_ssyn': Syn.ssyn[:],
#         'Syn_Isyns': Syn.Isyns[:],
#         'gsyn': Syn.gsyn[:],
#         'S_int_spikes': S_int.t[:],
#         'S_int_i': S_int.i[:],
#         'S_pyr_spikes': S_pyr.t[:],
#         'S_pyr_i': S_pyr.i[:],
#     }
#     mat.savemat(filename, data)

# def main():
#     """
#     Runs the simulation for different values of synaptic conductance and delay.
#     """
#     simtime = 180*second
#     pyr_Iapp, int_Iapp = input_current()
#     gsyn_list = [2,4,8]
#     delay_list = [0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5]
#     for gsyn in gsyn_list:
#         for delay in delay_list:
#             Pyr, Int, Syn, S_int, S_pyr = create_network(pyr_Iapp, int_Iapp, gsyn*mS, delay*ms, simtime)
#             save_name ="gsyn="+str(gsyn)+";delay="+str(delay)+".mat" 
#             print('saving data for gsyn=',gsyn,';delay=',delay)
#             # save_data(Pyr, Int, Syn, S_int, S_pyr, save_name)

#     return Pyr, Int, Syn, S_int, S_pyr

# if __name__ == "__main__":
#     Pyr, Int, Syn, S_int, S_pyr = main()
#     print("Simulation done")